In [1]:
%matplotlib inline
import zipline
from zipline.api import order_target_percent, symbol, \
    schedule_function, date_rules, time_rules
from datetime import datetime
import pytz
import pyfolio as pf
import matplotlib.pyplot as plt
import pandas as pd

In [1]:
securities = {
        'SPY': 0.25,
        'TLT': 0.3,
        'IEF': 0.3,
        'GLD': 0.075,
        'DBC': 0.075
    }

In [9]:
securities

{'DBC': 0.075, 'GLD': 0.075, 'IEF': 0.3, 'SPY': 0.25, 'TLT': 0.3}

In [10]:
securities.keys()

dict_keys(['TLT', 'IEF', 'SPY', 'DBC', 'GLD'])

In [8]:
securities.items()

dict_items([('TLT', 0.3), ('IEF', 0.3), ('SPY', 0.25), ('DBC', 0.075), ('GLD', 0.075)])

In [2]:
def initialize(context):
    # Securities and target weights
    context.securities = {
        'SPY': 0.25,
        'TLT': 0.3,
        'IEF': 0.3,
        'GLD': 0.075,
        'DBC': 0.075
    }
    
    # Schedule rebalance for once a month
    schedule_function(rebalance, date_rules.month_start(), time_rules.market_open())

In [3]:
def rebalance(context, data):
    # Loop through the securities
    for sec, weight in context.securities.items():
        sym = symbol(sec)
        # Check if we can trade
        if data.can_trade(sym):
            # Reset the weight
            order_target_percent(sym, weight)

In [4]:
def analyze(context, perf):
    # Use PyFolio to generate a performance report
    returns, portfolio, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)
    pf.create_returns_tear_sheet(returns, benchmark_rets = None)

In [5]:
# Set start and end date
start = datetime(1997, 1, 1, 8, 15, 12, 0, tzinfo = pytz.UTC)
end = datetime(2018, 12, 31, 8, 15, 12, 0, tzinfo = pytz.UTC)

In [6]:
# Fire off the backtest
result = zipline.run_algorithm(
    start = start,  # Set start
    end = end,  # Set end
    initialize = initialize,  # Defined startup function
    analyze = analyze, 
    capital_base = 10000,  # Set initial capital
    data_frequency = 'daily',  # Set data frequency
    bundle = 'ac_equities_db'  # Select bundle
)

UnknownBundle: No bundle registered with the name 'ac_equities_db'